In [1]:
import pandas as pd
import sklearn.feature_selection
import sklearn.metrics
import sklearn.model_selection
import sklearn.linear_model
import sklearn.covariance
import sklearn.cross_decomposition
import sklearn.feature_extraction
import sklearn.svm
import sklearn.compose

import statsmodels.api as sm
# from statsmodels.sandbox.regression.gmm import IV2SLS
from linearmodels.iv import IV2SLS

In [2]:
combo_df = pd.read_csv('combined.csv')


In [ ]:
# print(combo_df.dtypes)

In [3]:
df = combo_df.__deepcopy__()

# print(combo_df.isnull().sum().sum())
# print(121964*32)
df = df[df['Price'] >= 0]
df

,Unnamed: 0,Date,Zillow,MSA,CSA,CBSA,Total,1 Unit,2 Units,3 and 4 Units,...,Population 1,Population 5,Population Diff,Population,Pop Growth 1 Year,Pop -1 Years,Pop Percent 1 Year,Pop Growth 5 Year,Pop -5 Years,Pop Percent 5 Year
0,0,2019-11-30,"Abilene, TX","Abilene, TX",999.0,10180.0,23.0,21.0,2.0,0.0,...,171795.0,170669.0,0.006554,170669.0,660.0,170009.0,0.003882,3498.0,167171.0,0.020925
1,1,2019-12-31,"Abilene, TX","Abilene, TX",999.0,10180.0,37.0,31.0,6.0,0.0,...,171795.0,170669.0,0.006554,170669.0,660.0,170009.0,0.003882,3498.0,167171.0,0.020925
2,2,2020-01-31,"Abilene, TX","Abilene, TX",999.0,10180.0,29.0,29.0,0.0,0.0,...,NaN,171354.0,NaN,171354.0,685.0,170669.0,0.004014,3409.0,167945.0,0.020298
3,3,2020-02-29,"Abilene, TX","Abilene, TX",999.0,10180.0,30.0,30.0,0.0,0.0,...,NaN,171354.0,NaN,171354.0,685.0,170669.0,0.004014,3409.0,167945.0,0.020298
4,4,2020-03-31,"Abilene, TX","Abilene, TX",999.0,10180.0,29.0,27.0,2.0,0.0,...,NaN,171354.0,NaN,171354.0,685.0,170669.0,0.004014,3409.0,167945.0,0.020298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121959,121959,2022-03-31,"Yuma, AZ","Yuma, AZ",999.0,49740.0,89.0,83.0,6.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121960,121960,2022-04-30,"Yuma, AZ","Yuma, AZ",999.0,49740.0,66.0,66.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121961,121961,2022-05-31,"Yuma, AZ","Yuma, AZ",999.0,49740.0,88.0,84.0,4.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121962,121962,2022-06-30,"Yuma, AZ","Yuma, AZ",999.0,49740.0,65.0,65.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = df.dropna(axis = 'columns')
df

,Unnamed: 0,Date,Zillow,MSA,CSA,CBSA,Total,1 Unit,2 Units,3 and 4 Units,5 Units or More,Num of Structures With 5 Units or More,Year,Month,filename,Price
0,0,2019-11-30,"Abilene, TX","Abilene, TX",999.0,10180.0,23.0,21.0,2.0,0.0,0.0,0.0,2019,11,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...,81284.0
1,1,2019-12-31,"Abilene, TX","Abilene, TX",999.0,10180.0,37.0,31.0,6.0,0.0,0.0,0.0,2019,12,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...,81875.0
2,2,2020-01-31,"Abilene, TX","Abilene, TX",999.0,10180.0,29.0,29.0,0.0,0.0,0.0,0.0,2020,1,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...,82268.0
3,3,2020-02-29,"Abilene, TX","Abilene, TX",999.0,10180.0,30.0,30.0,0.0,0.0,0.0,0.0,2020,2,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...,82846.0
4,4,2020-03-31,"Abilene, TX","Abilene, TX",999.0,10180.0,29.0,27.0,2.0,0.0,0.0,0.0,2020,3,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...,84280.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121959,121959,2022-03-31,"Yuma, AZ","Yuma, AZ",999.0,49740.0,89.0,83.0,6.0,0.0,0.0,0.0,2022,3,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,357302.0
121960,121960,2022-04-30,"Yuma, AZ","Yuma, AZ",999.0,49740.0,66.0,66.0,0.0,0.0,0.0,0.0,2022,4,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,363285.0
121961,121961,2022-05-31,"Yuma, AZ","Yuma, AZ",999.0,49740.0,88.0,84.0,4.0,0.0,0.0,0.0,2022,5,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,370578.0
121962,121962,2022-06-30,"Yuma, AZ","Yuma, AZ",999.0,49740.0,65.0,65.0,0.0,0.0,0.0,0.0,2022,6,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month,376555.0


In [5]:
df['Date'] = pd.to_datetime(df['Date']).astype('int64')
df.dtypes

/var/folders/57/cc48k_bs6v9_xgql3_pmrykm0000gn/T/ipykernel_12309/3275139146.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date']).astype('int64')


Unnamed: 0                                  int64
Date                                        int64
Zillow                                     object
MSA                                        object
CSA                                       float64
CBSA                                      float64
Total                                     float64
1 Unit                                    float64
2 Units                                   float64
3 and 4 Units                             float64
5 Units or More                           float64
Num of Structures With 5 Units or More    float64
Year                                        int64
Month                                       int64
filename                                   object
Price                                     float64
dtype: object

In [6]:
df = df[['Date', 'CSA', 'CBSA', 'Total','1 Unit', '2 Units', '3 and 4 Units', '5 Units or More', 'Num of Structures With 5 Units or More', 'Year', 'Month', 'filename', 'Price']]
cols = df.columns.to_list()
cols.remove('filename')
scaler = sklearn.preprocessing.MinMaxScaler()
temp_df= df[['Date', 'CSA', 'CBSA', 'Total','1 Unit', '2 Units', '3 and 4 Units', '5 Units or More', 'Num of Structures With 5 Units or More', 'Year', 'Month', 'Price']]
# scaler.fit(temp_df)
temp_df = pd.DataFrame(scaler.fit_transform(temp_df))

temp_df.columns = cols
temp_df['filename'] = df['filename']

df = temp_df
#
df

,Date,CSA,CBSA,Total,1 Unit,2 Units,3 and 4 Units,5 Units or More,Num of Structures With 5 Units or More,Year,Month,Price,filename
0,0.000000,1.0,0.0,0.001935,0.003768,0.005236,0.0,0.000000,0.000000,0.000000,0.909091,0.021586,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...
1,0.031828,1.0,0.0,0.003112,0.005563,0.015707,0.0,0.000000,0.000000,0.000000,1.000000,0.021794,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...
2,0.063655,1.0,0.0,0.002439,0.005204,0.000000,0.0,0.000000,0.000000,0.333333,0.000000,0.021932,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...
3,0.093429,1.0,0.0,0.002524,0.005383,0.000000,0.0,0.000000,0.000000,0.333333,0.090909,0.022136,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...
4,0.125257,1.0,0.0,0.002439,0.004845,0.005236,0.0,0.000000,0.000000,0.333333,0.181818,0.022640,Metro_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.6...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120976,0.874743,1.0,1.0,0.007487,0.014893,0.015707,0.0,0.000000,0.000000,1.000000,0.181818,0.118650,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month
120977,0.905544,1.0,1.0,0.005552,0.011843,0.000000,0.0,0.000000,0.000000,1.000000,0.272727,0.120754,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month
120978,0.937372,1.0,1.0,0.007402,0.015073,0.010471,0.0,0.000000,0.000000,1.000000,0.363636,0.123318,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month
120979,0.968172,1.0,1.0,0.005468,0.011663,0.000000,0.0,0.000000,0.000000,1.000000,0.454545,0.125420,Metro_zhvi_uc_sfrcondo_tier_0.67_1.0_sm_sa_month


In [7]:
print(combo_df.columns)

Index(['Unnamed: 0', 'Date', 'Zillow', 'MSA', 'CSA', 'CBSA', 'Total', '1 Unit',
       '2 Units', '3 and 4 Units', '5 Units or More',
       'Num of Structures With 5 Units or More', 'Year', 'Month', 'filename',
       'Price', 'Seasonal', 'Trend', 'Residual', 'Price Change',
       'Trend Change', 'Year as Date', 'Population 1', 'Population 5',
       'Population Diff', 'Population', 'Pop Growth 1 Year', 'Pop -1 Years',
       'Pop Percent 1 Year', 'Pop Growth 5 Year', 'Pop -5 Years',
       'Pop Percent 5 Year'],
      dtype='object')


In [8]:
df['Single Unit ratio'] = df['1 Unit']/df['Total']

In [9]:
df['Single Unit ratio'].max()

2.133141934326216

In [10]:
df['Single Unit ratio'].min()

0.0

In [11]:
default_corr = df.corr(method = 'spearman')

/var/folders/57/cc48k_bs6v9_xgql3_pmrykm0000gn/T/ipykernel_12309/3247628189.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  default_corr = df.corr(method = 'spearman')


In [12]:
print(default_corr['Price'])

Date                                      0.175178
CSA                                       0.012707
CBSA                                      0.050128
Total                                     0.340217
1 Unit                                    0.313013
2 Units                                   0.207214
3 and 4 Units                             0.201758
5 Units or More                           0.327053
Num of Structures With 5 Units or More    0.329343
Year                                      0.166944
Month                                    -0.010101
Price                                     1.000000
Single Unit ratio                        -0.246364
Name: Price, dtype: float64


In [13]:
Pearson_corr = df.corr(method = 'pearson')
Pearson_corr

/var/folders/57/cc48k_bs6v9_xgql3_pmrykm0000gn/T/ipykernel_12309/1142935367.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Pearson_corr = df.corr(method = 'pearson')


,Date,CSA,CBSA,Total,1 Unit,2 Units,3 and 4 Units,5 Units or More,Num of Structures With 5 Units or More,Year,Month,Price,Single Unit ratio
Date,1.000000,0.003171,0.003696,0.043616,0.033344,0.033322,0.034044,0.045677,0.057083,0.941612,-0.065906,0.157400,-0.053179
CSA,0.003171,1.000000,0.142768,-0.130720,-0.140409,-0.085790,-0.094073,-0.089266,-0.111824,0.003049,-0.000395,0.004632,0.044234
CBSA,0.003696,0.142768,1.000000,-0.000400,-0.017396,0.024037,0.017938,0.018350,0.008637,0.003829,-0.001272,0.092112,0.018121
Total,0.043616,-0.130720,-0.000400,1.000000,0.915934,0.606227,0.468398,0.883566,0.807122,0.040494,-0.001173,0.192516,-0.286416
1 Unit,0.033344,-0.140409,-0.017396,0.915934,1.000000,0.491203,0.331671,0.622404,0.585281,0.034830,-0.012367,0.130537,-0.134048
2 Units,0.033322,-0.085790,0.024037,0.606227,0.491203,1.000000,0.444288,0.578591,0.610633,0.028716,0.005683,0.162456,-0.261475
3 and 4 Units,0.034044,-0.094073,0.017938,0.468398,0.331671,0.444288,1.000000,0.496228,0.542751,0.029497,0.005356,0.152831,-0.269006
5 Units or More,0.045677,-0.089266,0.018350,0.883566,0.622404,0.578591,0.496228,1.000000,0.886637,0.037936,0.012014,0.220785,-0.397585
Num of Structures With 5 Units or More,0.057083,-0.111824,0.008637,0.807122,0.585281,0.610633,0.542751,0.886637,1.000000,0.049981,0.007414,0.240054,-0.421002
Year,0.941612,0.003049,0.003829,0.040494,0.034830,0.028716,0.029497,0.037936,0.049981,1.000000,-0.398021,0.148700,-0.037764


In [ ]:
print(Pearson_corr['Price'])

In [ ]:
kendall_corr = df.corr(method = 'kendall')
kendall_corr

In [ ]:
print(kendall_corr['Price'])

In [14]:
linear_reg = sklearn.linear_model.LinearRegression()

ohe = sklearn.preprocessing.OneHotEncoder(sparse = False, handle_unknown='ignore')


In [15]:
df = df.dropna(axis = 'rows')
print(df.isnull().sum().sum())



0


In [16]:
X = df.drop(['Price'], axis = 'columns')
print(X.sample(5))
# print(X.head())
y = df[['Price']]
col_transform = sklearn.compose.make_column_transformer((ohe, ['filename']))

#
# X = pd.DataFrame(scaled, columns = X.columns)

col_transform.fit_transform(X)

print(col_transform)


           Date       CSA      CBSA     Total    1 Unit   2 Units  \
77841  0.156057  0.227933  0.446916  0.006309  0.010407  0.010471   
76953  0.439425  0.496089  0.379171  0.008159  0.017405  0.000000   
4166   0.687885  0.020112  0.338726  0.006309  0.013458  0.000000   
97327  0.530801  1.000000  0.052578  0.032806  0.068545  0.000000   
39615  0.842916  0.362011  0.250758  0.008496  0.018123  0.000000   

       3 and 4 Units  5 Units or More  Num of Structures With 5 Units or More  \
77841       0.026667         0.000487                                0.005025   
76953       0.000000         0.000000                                0.000000   
4166        0.000000         0.000000                                0.000000   
97327       0.026667         0.000000                                0.000000   
39615       0.000000         0.000000                                0.000000   

           Year     Month                                           filename  \
77841  0.333333  0

In [17]:
print(X)
print(y)

            Date  CSA  CBSA     Total    1 Unit   2 Units  3 and 4 Units  \
0       0.000000  1.0   0.0  0.001935  0.003768  0.005236            0.0   
1       0.031828  1.0   0.0  0.003112  0.005563  0.015707            0.0   
2       0.063655  1.0   0.0  0.002439  0.005204  0.000000            0.0   
3       0.093429  1.0   0.0  0.002524  0.005383  0.000000            0.0   
4       0.125257  1.0   0.0  0.002439  0.004845  0.005236            0.0   
...          ...  ...   ...       ...       ...       ...            ...   
120976  0.874743  1.0   1.0  0.007487  0.014893  0.015707            0.0   
120977  0.905544  1.0   1.0  0.005552  0.011843  0.000000            0.0   
120978  0.937372  1.0   1.0  0.007402  0.015073  0.010471            0.0   
120979  0.968172  1.0   1.0  0.005468  0.011663  0.000000            0.0   
120980  1.000000  1.0   1.0  0.014384  0.014534  0.000000            0.0   

        5 Units or More  Num of Structures With 5 Units or More      Year  \
0         

In [18]:

endog = df.columns.to_list()
endog.remove('Price')
endog.remove('filename')

x_const = sm.add_constant(df[endog])
first_model_results = sm.OLS(y, x_const, missing= 'Drop').fit()
first_model_results.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.107
Model:                            OLS   Adj. R-squared:                  0.107
Method:                 Least Squares   F-statistic:                     1262.
Date:                Mon, 17 Oct 2022   Prob (F-statistic):               0.00
Time:                        11:15:37   Log-Likelihood:             1.4612e+05
No. Observations:              115521   AIC:                        -2.922e+05
Df Residuals:                  115509   BIC:                        -2.921e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
const                                     -0.1264      0.096     -1.315      0.189      -0.315       0.062
Date                                      -0.6844      0.308     -2.223      0.026      -1.288      -0.081
CSA                                        0.0042      0.001      7.333      0.000       0.003       0.005
CBSA                                       0.0228      0.001     32.205      0.000       0.021       0.024
Total                                      0.0123      0.005      2.326      0.020       0.002       0.023
1 Unit                                     0.0011      0.004      0.237      0.813      -0.008       0.010
2 Units                                    0.0216      0.006      3.565      0.000       0.010       0.033
3 and 4 Units                              0.0269      0.006      4.804      0.000       0.016       0.038
5 Units or More                            0.0121      0.008      1.521      0.128      -0.004       0.028
Num of Structures With 5 Units or More     0.1698      0.009     19.859      0.000       0.153       0.187
Year                                       0.8062      0.346      2.329      0.020       0.128       1.485
Month                                      0.2454      0.106      2.315      0.021       0.038       0.453
Single Unit ratio                         -0.0230      0.000    -53.493      0.000      -0.024      -0.022
==============================================================================
Omnibus:                    79335.217   Durbin-Watson:                   0.090
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1703460.005
Skew:                           3.039   Prob(JB):                         0.00
Kurtosis:                      20.803   Cond. No.                     1.04e+17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.64e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [20]:




# endog = df.columns.to_list()
# endog.remove('Price')
# endog.remove('filename')
print(endog)
for end in endog:
    print(end)
    print(len(df[end]))

iv2sls_model = IV2SLS(df['Price'], df[endog], None, None ).fit(cov_type="unadjusted")
print(iv2sls_model)

['Date', 'CSA', 'CBSA', 'Total', '1 Unit', '2 Units', '3 and 4 Units', '5 Units or More', 'Num of Structures With 5 Units or More', 'Year', 'Month', 'Single Unit ratio']
Date
115521
CSA
115521
CBSA
115521
Total
115521
1 Unit
115521
2 Units
115521
3 and 4 Units
115521
5 Units or More
115521
Num of Structures With 5 Units or More
115521
Year
115521
Month
115521
Single Unit ratio
115521


ValueError: regressors [exog endog] do not have full column rank

In [ ]:


pipe = sklearn.pipeline.make_pipeline(col_transform, linear_reg)

In [ ]:
sklearn.metrics.get_scorer_names()

In [ ]:
for scoring in sklearn.metrics.get_scorer_names():
    print(scoring)
    try:
        sklearn.model_selection.cross_val_score(pipe, X, y, cv=5, scoring = scorer).mean()
    except: continue

In [ ]:
# ind_vars = ['CSA', 'CBSA', 'Total', '1 Unit','2 Units', '3 and 4 Units', '5 Units or More',
#             'Num of Structures With 5 Units or More', 'Year', 'Month', 'Population 1', 'Population 5',
#             'Population Diff','Population', 'Pop Growth 1 Year', 'Pop -1 Years','Pop Percent 1 Year',
#             'Pop Growth 5 Year',
#             'Pop -5 Years','Pop Percent 5 Year', 'Single Unit ratio']
# dep_vars = ['Price']
#
# cols = ['CSA', 'CBSA', 'Total', '1 Unit','2 Units', '3 and 4 Units', '5 Units or More',
#         'Num of Structures With 5 Units or More', 'Year', 'Month', 'Population 1', 'Population 5',
#         'Population Diff','Population', 'Pop Growth 1 Year', 'Pop -1 Years','Pop Percent 1 Year',
#         'Pop Growth 5 Year','Pop -5 Years','Pop Percent 5 Year', 'Single Unit ratio', 'Price']
#
# # print(cols)
# no_null_df = combo_df[cols].dropna()
# # print(no_null_df.head())
# x = no_null_df[ind_vars]
# y = no_null_df[dep_vars]
# #
# print(x.head())
#
# x_const = sm.add_constant(x)
#
# first_model_results = sm.OLS(y, x_const).fit_regularized()
# first_model_results.summary()